In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import sys
from sklearn.neighbors import KNeighborsClassifier  
import matplotlib.pyplot as plt
import networkx as nx
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2
from sklearn.tree import DecisionTreeClassifier
import time
from db.database import DatabaseConnector
from dtloader.dataloader import DataLoader
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.stattools import grangercausalitytests
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *
import seaborn as sns
from granger.grangercausality_reg import GrangerCausalityTest
from granger.grangercausality_cl import GrangerCausalityDiscrete
from segmentation.segmentation import *
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.tsatools import lagmat2ds
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from utils.features_eng import *
from scipy import stats
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.stattools import grangercausalitytests

In [2]:
#connecting to the database 
dl = DataLoader()
dl.load('../sample_testing/sample2.log')
dl.extractinfo(export=False,single_file=False)

mongodb://localhost:27017/
Database connected


C:\Users\PC\Desktop\TUM Summer 2019\Drone DC\DroneCrashDetection\dtloader\dataloader.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.df[3] = self.df[3].str.strip() # cleaning third column


In [3]:
#dl.dbconnector.query_str('sample2')

In [4]:
pd.DataFrame(dl.dbconnector.query('GPS_sample_testing/sample2')).head()

,_id,TimeUS,Status,GMS,GWk,NSats,HDop,Lat,Lng,RAlt,Alt,Spd,GCrs,VZ,GPS_0,lineIndex
0,5cfd4a85048d2b431094bd28,216191643,3,66089600,1950,7,1.28,56.590823,37.201885,0.00,123.81,0.11,328.3,-0.22,1,611
1,5cfd4a85048d2b431094bd28,216212018,3,66090200,1950,7,1.28,56.590823,37.201885,0.00,123.81,0.03,328.3,-0.15,1,616
2,5cfd4a85048d2b431094bd28,216271200,3,66090200,1950,7,1.28,56.590821,37.201886,0.01,124.23,0.06,328.3,0.03,1,621
3,5cfd4a85048d2b431094bd28,216471200,3,66092200,1950,7,1.28,56.590821,37.201886,0.00,124.28,0.09,328.3,0.03,1,669
4,5cfd4a85048d2b431094bd28,216671445,3,66092400,1950,7,1.28,56.590821,37.201886,-0.04,124.30,0.04,328.3,-0.10,1,719


## Granger Causality Sample Testing

For this test we need to put the two variables in one table with the line indexes and then test everything 

In [19]:
x = 'CURR_Volt'
y = 'ATT_Pitch'
z = 'CTUN_ThrOut'

In [20]:
def signal_comp_va(signal_name):
    return  signal_name[:signal_name.index('_')],signal_name[signal_name.index('_')+1:]

def ft_eg(signal_name,sr,segmts,lag=5):
    """
    given a continuous series return a dataframe with added features
    methods are implemented in the file feature_eng.py in the utils directory
    """
    seg_ratio = segment_ratio(sr, segmts)
    seg_means = segment_mean(sr,segmts)
    seg_ratio_perc = segment_ratio_outlier(seg_ratio)
    moving_avg_data = moving_average_window(sr, 10)
    lagged_segments = lagged_segment(sr, segmts) #lagged segment ratios
    lagged_percentile = seg_ratio_perc.shift(lag).fillna(method='bfill')
    lagged_seg_means = seg_means.shift(lag).fillna(method='bfill')
    lagged_mov_avg = moving_avg_data.shift(lag).fillna(method='bfill')
    
    dataset = pd.DataFrame()
    #dataset['seg_ratio'] = seg_ratio
    #dataset['seg_ratio_percentile'] = seg_ratio_perc
    #dataset[signal_name+' lagged_moving_avg'] = moving_avg_data
    dataset[signal_name+' lagged_segments'] = lagged_segments
    dataset[signal_name+' lagged_perc'] = lagged_percentile
    dataset[signal_name+' lagged_means'] = lagged_seg_means
    return dataset

def segment_signal(comp,va,dl,filename):
    """
    return a segment list of the signal
    """
    sg = Segmentation(1)
    fitmethod= 'inter'
    sgmethod = 'td'
    sr = dl.dbconnector.query(comp+'_' + filename,va)
    sig_length = len(sr)
    rng = dl.dbconnector.query(comp+'_'+filename,'lineIndex')
    return sg.segment(sr,seg_method=sgmethod,fit_method=fitmethod,err_growth=0.2,batch=True,batch_size=50)


def align_index(filename,comp_x,va_x,comp_z,va_z):
    #aligning the time index of the two signals
    #---------------------------------------------------------------------------
    ts = {comp_x:[va_x],comp_z:[va_z]}
    dft = corr_var(filename,dl,ts,find_corr=False)
    dft.reset_index(inplace=True)
    dft.drop('lineIndex',axis=1,inplace=True)
    dft.dropna(axis=0,how='any',inplace=True) #remove nans
    dft.index = range(len(dft))
    #for variables with the same keys we add the other manaully    
    #to be fixed later
    if len(dft.columns) == 1:
        #same key case - add the other to the dataframe
        dft[comp_x+'_'+va_x] = dl.dbconnector.query(comp_x+'_'+filename,va_x)
    return dft

In [21]:
filename = 'sample_testing/sample2'


def causality_cls_preprocess(filename, lag):
    """
    preprocess data to put it in the write form for granger test using classification

    :param filename : log file to process
    :param lag: lag value for features 

    """
    # segmenting signals
    # not loading from database to test different segments with max error and batch sizes
    # ------------------------------------------------------------------------------------
    # segment x
    comp_x, va_x = signal_comp_va(x)
    seg_x = segment_signal(comp_x, va_x, dl, filename)

    # segment y
    comp_y, va_y = signal_comp_va(y)
    seg_y = segment_signal(comp_y, va_y, dl, filename)

    # segment z
    comp_z, va_z = signal_comp_va(z)
    seg_z = segment_signal(comp_z, va_z, dl, filename)

    # align time index between two files
    dft = align_index(filename, comp_x, va_x, comp_z, va_z)
    dft_y = align_index(filename, comp_y, va_y, comp_z, va_z)

    # datasets containing the features for predictions
    # Adding features
    # ---------------------------------------------------------------------------
    dataset_x = ft_eg(comp_x+'_'+va_x, dft[comp_x+'_'+va_x], seg_x, lag=lag)
    dataset_y = ft_eg(comp_y+'_'+va_y, dft_y[comp_y+'_'+va_y], seg_y, lag=lag)
    dataset_z = ft_eg(comp_z+'_'+va_z, dft[comp_z+'_'+va_z], seg_z, lag=lag)

    # Building the target variable for z
    # ---------------------------------------------------------------------------
    seg_ratio_z = segment_ratio(dft[comp_z+'_'+va_z], seg_z)
    target = np.zeros_like(seg_ratio_z)
    threshold_event_occurred = -0.01  # Event = Massive  drop
    idx = event_finder(seg_ratio_z, threshold_event_occurred, "lesser")
    target[idx] = -1
    threshold_event_occurred = 0.01  # Event = Massive  rise
    idx = event_finder(seg_ratio_z, threshold_event_occurred, "greater")
    target[idx] = 1
    
    
    #if another variable was used the segments will not be equal to the lineindex concat so another must be constructed
    seg_ratio_z = segment_ratio(dft_y[comp_z+'_'+va_z], seg_z)
    target_y = np.zeros_like(seg_ratio_z)
    threshold_event_occurred = -0.01  # Event = Massive  drop
    idx = event_finder(seg_ratio_z, threshold_event_occurred, "lesser")
    target_y[idx] = -1
    threshold_event_occurred = 0.01 # Event = Massive  rise
    idx = event_finder(seg_ratio_z, threshold_event_occurred, "greater")
    target_y[idx] = 1

    # combining features for full model
    # ---------------------------------------------------------------------------
    dataset_combined_xz = pd.concat([dataset_x, dataset_z], axis=1).dropna()
    dataset_combined_yz = pd.concat([dataset_y, dataset_z], axis=1).dropna()
    return dft, dft_y, dataset_x, dataset_y, dataset_z, dataset_combined_xz, dataset_combined_yz, target,target_y


def test_causality(df, comp_x, va_x, dataset_red, dataset_combined, comp_z, va_z, target,cv):
    l = ['nb', 'dt', 'knn', 'lr']
    gcl = GrangerCausalityDiscrete(
        names=[comp_x+'_'+va_x, comp_x+' +' + comp_z])
    dict_cls = {}
    for cls in l:
        res = gcl.test(df[comp_x+'_'+va_x], df[comp_z+'_'+va_z], dataset_red, dataset_combined, target, check_stationary=False,
                       classifier=cls, verbose=False, cv=cv)
        dict_cls[cls] = {'Mean-Acc-Full': gcl.mean_full,
                         'Mean-Acc-Reduced': gcl.mean_reduced, 'P-Value': gcl.pvalue}
    return dict_cls

# X Causes Z

In [31]:
lag = 500
cv = 2
dft_x, dft_y, dataset_x, dataset_y, dataset_z, dataset_combined_xz, dataset_combined_yz, target_x, target_y = causality_cls_preprocess(
    filename, lag)
comp_y, va_y = signal_comp_va(y)
comp_z, va_z = signal_comp_va(z)
comp_x, va_x = signal_comp_va(x)


dict_cls =  test_causality(dft_x,comp_x,va_x,dataset_z,dataset_combined_xz,comp_z,va_z,target_x,cv=cv)
resdf = pd.DataFrame(dict_cls).T
print('Mean Across Full Classifiers : ',resdf['Mean-Acc-Full'].mean())
print('Mean Across Reduced Classifiers : ',resdf['Mean-Acc-Reduced'].mean())
resdf

C:\Users\PC\Desktop\TUM Summer 2019\Drone DC\DroneCrashDetection\segmentation\segmentation.py:252: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (p,residuals,_,_) = lstsq(A,y)


Mean Across Full Classifiers :  0.6041831366879853
Mean Across Reduced Classifiers :  0.5824163871158087


,Mean-Acc-Full,Mean-Acc-Reduced,P-Value
nb,0.680815,0.685005,0.990663
dt,0.571788,0.511419,0.830310
knn,0.555579,0.567881,0.965744
lr,0.608550,0.565360,0.869024


# Y Causes Z

In [32]:
dict_cls =  test_causality(dft_y,comp_y,va_y,dataset_z,dataset_combined_yz,comp_z,va_z,target_y,cv=cv)
resdf = pd.DataFrame(dict_cls).T
print('Mean Across Full Classifiers : ',resdf['Mean-Acc-Full'].mean())
print('Mean Across Reduced Classifiers : ',resdf['Mean-Acc-Reduced'].mean())
resdf

Mean Across Full Classifiers :  0.5754969271413397
Mean Across Reduced Classifiers :  0.5824163871158087


,Mean-Acc-Full,Mean-Acc-Reduced,P-Value
nb,0.649082,0.685005,0.910034
dt,0.511419,0.511419,1.000000
knn,0.569838,0.567881,0.994816
lr,0.571649,0.565360,0.979004


In [30]:
# d = pd.DataFrame()
# comp_x, va_x = signal_comp_va(x)
# comp_y, va_y = signal_comp_va(y)
# d[y] = dl.dbconnector.query(comp_y+'_sample_testing/sample2',va_y)
# d[z] = dl.dbconnector.query(comp_z+'_sample_testing/sample2',va_z)
# grangercausalitytests(np.array(d),maxlag=10,verbose=1)[10]

In [ ]:
#Testing on datasets
clf = DecisionTreeClassifier(max_depth=6)
scores_red= cross_val_score(clf,dataset_z,target_x, cv=cv,scoring='accuracy')
scores_full= cross_val_score(clf,dataset_combined_xz,target_x, cv=cv,scoring='accuracy')
print('full',np.mean(scores_full))
print('red',np.mean(scores_red))

In [ ]:
dataset_z.head()

In [ ]:
dataset_combined_xz.head()